In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql
plt.style.use('fivethirtyeight')
pd.options.display.float_format = '{:,.2f}'.format

In [86]:
df_sic = pd.read_csv('../data/DEBITO_SIC.csv', decimal=',', thousands='.',
                    names=['Fatura','Tipo','ID','Valor'], skiprows=1)
df_vertica = pd.read_csv('../data/DEBITO_VERTICA.csv', decimal=',', thousands='.',
                        names=['Fatura','Tipo','ID','Valor'], skiprows=1)

In [87]:
df_mrg = pd.merge(df_vertica, df_sic,how='left', 
                  on="Fatura", suffixes=('_sei', '_sic'))

In [89]:
df_mrg['Valor_sei'] = df_mrg['Valor_sei'].replace(np.nan , 0)
df_mrg['Valor_sic'] = df_mrg['Valor_sic'].replace(np.nan , 0)
df_mrg['diferenca'] = df_mrg['Valor_sei'] - df_mrg['Valor_sic']

In [90]:
sum(df_mrg['diferenca'])

442.9599999999999

In [93]:
df_mrg[df_mrg['diferenca'] != 0].to_csv('Divergencias_sei_sic.csv')

## CONSULTAS 

```sql
-------------------------------------------------
-------------------------------------------------
-- CONFERENCIA PRAX
-- VALOR DE DEBITO PARA COMPARAÇÃO
-------------------------------------------------
-------------------------------------------------
SELECT
	fat_seq_fatura,
	CASE
		WHEN con_seq_contrato is null and cli_seq_cliente is not null 
			THEN 'CLIENTE'
		WHEN con_seq_contrato is not null 
			THEN 'CONTRATO'
		ELSE 'VERIFICAR'
	END AS tipo_debito,
	CASE
		WHEN con_seq_contrato is null and cli_seq_cliente is not null 
			THEN cli_seq_cliente
		WHEN con_seq_contrato is not null 
			THEN con_seq_contrato
		ELSE 0
	END AS id_cli_contrato,
	sum(VLR_ITEM_FATURA) VLR_ITEM_FATURA
FROM 
	dbo.BASE_DEBITOS_PRAX
group by 
	fat_seq_fatura,
	con_seq_contrato,
	cli_seq_cliente
order by 
	fat_seq_fatura;
```

```sql
-------------------------------------------------
-------------------------------------------------
-- CONFERENCIA FATO
-- VALOR DE DEBITO PARA COMPARAÇÃO
-------------------------------------------------
-------------------------------------------------
SELECT
	fat_seq_fatura,
	CASE
		WHEN con_seq_contrato is null and cli_seq_cliente is not null 
			THEN 'CLIENTE'
		WHEN con_seq_contrato is not null 
			THEN 'CONTRATO'
		ELSE 'VERIFICAR'
	END AS tipo_debito,
	CASE
		WHEN con_seq_contrato is null and cli_seq_cliente is not null 
			THEN cli_seq_cliente
		WHEN con_seq_contrato is not null 
			THEN con_seq_contrato
		ELSE 0
	END AS id_cli_contrato,
	sum(vlr_debito_item_fatura)
FROM 
	dw.fat_deb_debito_diario
group by 
	fat_seq_fatura,
	con_seq_contrato,
	cli_seq_cliente
order by 
	fat_seq_fatura
```